In [1]:
import pandas as pd
import random
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split # 학습, 테스트set 구분
from sklearn.tree import export_graphviz # tree 시각화를 위해
import graphviz # tree 시각화
from sklearn.metrics import f1_score# 성능지표를 계산하기 위해 import
from sklearn.model_selection import cross_val_score, cross_validate # 교차검증

In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/LGaimers/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/LGaimers/test.csv')

In [3]:
print(train_df.shape)
print(test_df.shape)

(598, 2881)
(310, 2879)


In [4]:
# 모두 결측치인 변수 제거
def remove_all_nan(dataframe):
  col_list = dataframe.columns
  nan_list = []
  nan_cnt = []
  nan_col = []
  full_list = []

  for col in col_list:
    if dataframe[col].isnull().sum() == 0:
      full_list.append(col)
      continue
    nan_list.append([col,dataframe[col].isnull().sum()])
    nan_cnt.append(dataframe[col].isnull().sum())
    nan_col.append(col)

  del_col = []
  for nan in nan_list:
    if nan[1] == len(dataframe):
      del_col.append(nan[0])
  
  return dataframe.drop(columns=del_col)

# LINE별 dataset 구축

In [5]:
train_T050304 = train_df[train_df['LINE'] == 'T050304']
train_T050307 = train_df[train_df['LINE'] == 'T050307']
train_T100304 = train_df[train_df['LINE'] == 'T100304']
train_T100306 = train_df[train_df['LINE'] == 'T100306']
train_T010306 = train_df[train_df['LINE'] == 'T010306']
train_T010305 = train_df[train_df['LINE'] == 'T010305']

In [6]:
test_T050304 = test_df[test_df['LINE'] == 'T050304']
test_T050307 = test_df[test_df['LINE'] == 'T050307']
test_T100304 = test_df[test_df['LINE'] == 'T100304']
test_T100306 = test_df[test_df['LINE'] == 'T100306']
test_T010306 = test_df[test_df['LINE'] == 'T010306']
test_T010305 = test_df[test_df['LINE'] == 'T010305']

test set에 존재하는 모두 nan인 변수에 대해 -1로 처리하는 것과 없애는 것 중 뭐가 좋은지는 논의해봐야할 것 같습니다.
  - 일단 없애지않고 진행하겠습니다.

In [7]:
print('train T050304 :' ,train_T050304['PRODUCT_CODE'].unique(), 'test T050304 :' ,test_T050304['PRODUCT_CODE'].unique())
print('train T050307 :' ,train_T050307['PRODUCT_CODE'].unique(), 'test T050307 :' ,test_T050307['PRODUCT_CODE'].unique())
print('train T100304 :' ,train_T100304['PRODUCT_CODE'].unique(), 'test T100304 :' ,test_T100304['PRODUCT_CODE'].unique())
print('train T100306 :' ,train_T100306['PRODUCT_CODE'].unique(), 'test T100306 :' ,test_T100306['PRODUCT_CODE'].unique())
print('train T010306 :' ,train_T010306['PRODUCT_CODE'].unique(), 'test T010306 :' ,test_T010306['PRODUCT_CODE'].unique())
print('train T010305 :' ,train_T010305['PRODUCT_CODE'].unique(), 'test T010305 :' ,test_T010305['PRODUCT_CODE'].unique())


train T050304 : ['A_31'] test T050304 : ['A_31']
train T050307 : ['A_31'] test T050307 : ['A_31']
train T100304 : ['T_31' 'O_31'] test T100304 : ['T_31' 'O_31']
train T100306 : ['T_31' 'O_31'] test T100306 : ['T_31' 'O_31']
train T010306 : ['A_31'] test T010306 : ['A_31']
train T010305 : ['A_31'] test T010305 : ['A_31']


In [8]:
# A 생성 LINE
train_T050304, test_T050304
train_T050307, test_T050307
train_T010306, test_T010306
train_T010305, test_T010305

# T, O 생성 LINE
train_T100304, test_T100304
train_T100306, test_T100306

(    PRODUCT_ID  Y_Class  Y_Quality         TIMESTAMP     LINE PRODUCT_CODE  \
 26   TRAIN_026        1   0.531992   2022-06-19 9:20  T100306         T_31   
 30   TRAIN_030        1   0.528646  2022-06-19 23:39  T100306         T_31   
 36   TRAIN_036        1   0.529633   2022-06-22 3:39  T100306         T_31   
 42   TRAIN_042        1   0.530965   2022-06-23 3:39  T100306         T_31   
 43   TRAIN_043        1   0.529498   2022-06-23 6:28  T100306         T_31   
 ..         ...      ...        ...               ...      ...          ...   
 589  TRAIN_589        1   0.529510  2022-09-06 18:00  T100306         T_31   
 590  TRAIN_590        1   0.529948   2022-09-07 1:01  T100306         T_31   
 591  TRAIN_591        1   0.529308   2022-09-07 1:09  T100306         T_31   
 593  TRAIN_593        1   0.526546  2022-09-08 14:30  T100306         T_31   
 597  TRAIN_597        1   0.533702  2022-09-08 14:46  T100306         O_31   
 
       X_1   X_2  X_3   X_4  ...  X_2866  X_2867  

## LINE별 train set은 최대한 구축 = 모두 NaN이 아니라면 mean을 적용
  - 단 y_class별로 나눠서 적용하는게 합리적인듯

PRODUCT_CODE A

Y_Class == 0

In [9]:
train_T050304_0 = train_T050304[train_T050304['Y_Class'] == 0]
train_T050304_0 = train_T050304_0.fillna(train_T050304_0.median(numeric_only=True))
train_T050304_0

PRODUCT_ID  Y_Class  Y_Quality         TIMESTAMP     LINE PRODUCT_CODE  \
34   TRAIN_034        0   0.521524  2022-06-21 17:36  T050304         A_31   
109  TRAIN_109        0   0.516711  2022-07-06 19:15  T050304         A_31   
110  TRAIN_110        0   0.519451  2022-07-06 19:23  T050304         A_31   
111  TRAIN_111        0   0.518960  2022-07-06 20:41  T050304         A_31   
131  TRAIN_131        0   0.519810   2022-07-10 0:57  T050304         A_31   
132  TRAIN_132        0   0.521259   2022-07-10 1:05  T050304         A_31   
202  TRAIN_202        0   0.521544   2022-07-21 4:53  T050304         A_31   
255  TRAIN_255        0   0.515083  2022-07-25 12:39  T050304         A_31   
322  TRAIN_322        0   0.522300  2022-07-31 22:37  T050304         A_31   
344  TRAIN_344        0   0.524567   2022-08-03 8:28  T050304         A_31   
357  TRAIN_357        0   0.520856  2022-08-04 14:53  T050304         A_31   
360  TRAIN_360        0   0.524606  2022-08-04 21:17  T050304         A_31   
361  TRAIN_361        0   0.517494   2022-08-05 0:10  T050304         A_31   
362  TRAIN_362        0   0.515229   2022-08-05 0:18  T050304         A_31   
363  TRAIN_363        0   0.515892   2022-08-05 0:26  T050304         A_31   
367  TRAIN_367        0   0.525046   2022-08-05 7:10  T050304         A_31   
370  TRAIN_370        0   0.519903  2022-08-05 10:16  T050304         A_31   
375  TRAIN_375        0   0.521051  2022-08-05 22:48  T050304         A_31   
384  TRAIN_384        0   0.519332   2022-08-07 4:19  T050304         A_31   
385  TRAIN_385        0   0.521133   2022-08-07 4:27  T050304         A_31   
453  TRAIN_453        0   0.521341  2022-08-14 12:22  T050304         A_31   
478  TRAIN_478        0   0.520252  2022-08-16 12:20  T050304         A_31   
540  TRAIN_540        0   0.513751   2022-08-25 7:44  T050304         A_31   
579  TRAIN_579        0   0.521987   2022-09-05 5:59  T050304         A_31   
580  TRAIN_580        0   0.522460   2022-09-05 6:07  T050304         A_31   
581  TRAIN_581        0   0.523465   2022-09-05 8:18  T050304         A_31   
582  TRAIN_582        0   0.522233   2022-09-05 8:26  T050304         A_31   
583  TRAIN_583        0   0.522340   2022-09-05 8:34  T050304         A_31   
594  TRAIN_594        0   0.524022  2022-09-08 22:38  T050304         A_31   
595  TRAIN_595        0   0.521289  2022-09-08 22:47  T050304         A_31   

     X_1  X_2  X_3  X_4  ...  X_2866  X_2867  X_2868  X_2869  X_2870  X_2871  \
34   NaN  NaN  NaN  NaN  ...  55.030   52.24   55.33   57.49  67.310     1.0   
109  NaN  NaN  NaN  NaN  ...  56.860   50.46   40.49   57.34  66.370     1.0   
110  NaN  NaN  NaN  NaN  ...  48.280   51.25   55.43   56.95  67.440     1.0   
111  NaN  NaN  NaN  NaN  ...  54.115   54.71   52.51   56.30  65.035     1.0   
131  NaN  NaN  NaN  NaN  ...  52.970   59.15   42.29   57.22  65.460     1.0   
132  NaN  NaN  NaN  NaN  ...  54.115   54.71   52.51   56.30  65.035     1.0   
202  NaN  NaN  NaN  NaN  ...  54.115   54.71   52.51   56.30  65.035     1.0   
255  NaN  NaN  NaN  NaN  ...  60.240   56.16   54.52   56.42  66.850     1.0   
322  NaN  NaN  NaN  NaN  ...  55.990   52.08   40.90   55.31  62.800     1.0   
344  NaN  NaN  NaN  NaN  ...  54.630   59.05   40.77   55.24  64.690     1.0   
357  NaN  NaN  NaN  NaN  ...  54.530   59.33   50.85   55.33  65.870     1.0   
360  NaN  NaN  NaN  NaN  ...  48.400   60.71   53.14   49.39  62.830     1.0   
361  NaN  NaN  NaN  NaN  ...  54.650   62.37   51.43   55.55  64.630     1.0   
362  NaN  NaN  NaN  NaN  ...  55.170   60.58   41.07   58.09  65.240     1.0   
363  NaN  NaN  NaN  NaN  ...  53.700   54.98   56.42   58.47  63.770     1.0   
367  NaN  NaN  NaN  NaN  ...  52.390   57.23   36.81   51.47  65.140     1.0   
370  NaN  NaN  NaN  NaN  ...  53.690   52.46   52.58   56.62  66.490     1.0   
375  NaN  NaN  NaN  NaN  ...  55.540   52.66   59.68   52.99  63.160     1.0   
384  NaN  NaN  NaN  NaN  ...  54.900   61.30   54.06   59.93  62.

Y_Class == 1

In [10]:
train_T050304_1 = train_T050304[train_T050304['Y_Class'] == 1]
train_T050304_1 = train_T050304_1.fillna(train_T050304_1.median(numeric_only=True))
train_T050304_1

PRODUCT_ID  Y_Class  Y_Quality         TIMESTAMP     LINE PRODUCT_CODE  \
0    TRAIN_000        1   0.533433   2022-06-13 5:14  T050304         A_31   
2    TRAIN_002        1   0.531267   2022-06-13 5:30  T050304         A_31   
4    TRAIN_004        1   0.531590   2022-06-13 5:47  T050304         A_31   
6    TRAIN_006        1   0.533665   2022-06-13 6:03  T050304         A_31   
8    TRAIN_008        1   0.531821   2022-06-13 6:19  T050304         A_31   
10   TRAIN_010        1   0.531503   2022-06-13 6:36  T050304         A_31   
12   TRAIN_012        1   0.533632   2022-06-13 6:52  T050304         A_31   
14   TRAIN_014        1   0.531254   2022-06-13 7:08  T050304         A_31   
16   TRAIN_016        1   0.534054   2022-06-13 7:25  T050304         A_31   
18   TRAIN_018        1   0.532170   2022-06-13 7:41  T050304         A_31   
20   TRAIN_020        1   0.531179   2022-06-13 7:58  T050304         A_31   
62   TRAIN_062        1   0.526346  2022-06-25 17:20  T050304         A_31   
63   TRAIN_063        1   0.528817  2022-06-25 17:29  T050304         A_31   
96   TRAIN_096        1   0.530167  2022-07-03 18:21  T050304         A_31   
97   TRAIN_097        1   0.531679  2022-07-03 18:29  T050304         A_31   
199  TRAIN_199        1   0.527398  2022-07-20 23:31  T050304         A_31   
249  TRAIN_249        1   0.525494   2022-07-25 6:20  T050304         A_31   
250  TRAIN_250        1   0.525095   2022-07-25 6:28  T050304         A_31   
297  TRAIN_297        1   0.527887   2022-07-31 0:54  T050304         A_31   
303  TRAIN_303        1   0.529130   2022-07-31 4:50  T050304         A_31   
304  TRAIN_304        1   0.526559   2022-07-31 4:58  T050304         A_31   
320  TRAIN_320        1   0.526868  2022-07-31 18:58  T050304         A_31   
321  TRAIN_321        1   0.529237  2022-07-31 19:06  T050304         A_31   
323  TRAIN_323        1   0.529035   2022-08-01 2:45  T050304         A_31   
335  TRAIN_335        1   0.528340   2022-08-03 1:12  T050304         A_31   
337  TRAIN_337        1   0.530900   2022-08-03 3:30  T050304         A_31   
338  TRAIN_338        1   0.528027   2022-08-03 7:16  T050304         A_31   
340  TRAIN_340        1   0.530202  2022-08-03 14:20  T050304         A_31   
342  TRAIN_342        1   0.532929  2022-08-03 14:28  T050304         A_31   
346  TRAIN_346        1   0.531492   2022-08-03 8:37  T050304         A_31   
366  TRAIN_366        1   0.528552   2022-08-05 4:49  T050304         A_31   
368  TRAIN_368        1   0.528186   2022-08-05 8:36  T050304         A_31   
433  TRAIN_433        1   0.527332   2022-08-12 8:44  T050304         A_31   
573  TRAIN_573        1   0.526948  2022-09-03 20:03  T050304         A_31   
574  TRAIN_574        1   0.529416  2022-09-03 20:11  T050304         A_31   
575  TRAIN_575        1   0.526033  2022-09-03 20:27  T050304         A_31   
576  TRAIN_576        1   0.529681  2022-09-03 20:36  T050304         A_31   

     X_1  X_2  X_3  X_4  ...  X_2866  X_2867  X_2868  X_2869  X_2870  X_2871  \
0    NaN  NaN  NaN  NaN  ...   39.34   40.89  32.560   34.09   77.77     1.0   
2    NaN  NaN  NaN  NaN  ...   39.19   36.65  42.470   36.53   78.35     1.0   
4    NaN  NaN  NaN  NaN  ...   38.70   41.89  46.930   33.09   76.97     1.0   
6    NaN  NaN  NaN  NaN  ...   32.50   41.42  38.360   30.83   76.93     1.0   
8    NaN  NaN  NaN  NaN  ...   42.25   43.17  55.600   33.26   78.50     1.0   
10   NaN  NaN  NaN  NaN  ...   32.12   34.74  54.930   33.16   76.30     1.0   
12   NaN  NaN  NaN  NaN  ...   36.55   32.32  54.250   35.15   78.86     1.0   
14   NaN  NaN  NaN  NaN  ...   40.42   38.33  40.140   30.72   76.52     1.0   
16   NaN  NaN  NaN  NaN  ...   40.53   31.70  49.490   35.73   79.75     1.0   
18   NaN  NaN  NaN  NaN  ...   35.07   48.38  36.240   33.00   75.82     1.0   
20   NaN  NaN  NaN  NaN  ...   54.11   55.87  50.765   55.43   65.20     1.0   
62   NaN  NaN  NaN  NaN  ...   57.75   65.48  52.270   56.54   64.35     1.0   

Y_Class == 2

In [11]:
train_T050304_2 = train_T050304[train_T050304['Y_Class'] == 2]
train_T050304_2 = train_T050304_2.fillna(train_T050304_2.median(numeric_only=True))
train_T050304_2

PRODUCT_ID  Y_Class  Y_Quality         TIMESTAMP     LINE PRODUCT_CODE  \
21   TRAIN_021        2   0.544129   2022-06-14 5:58  T050304         A_31   
79   TRAIN_079        2   0.538629   2022-07-01 0:54  T050304         A_31   
80   TRAIN_080        2   0.537733   2022-07-01 1:11  T050304         A_31   
188  TRAIN_188        2   0.535405   2022-07-19 6:27  T050304         A_31   
314  TRAIN_314        2   0.537046  2022-07-31 14:15  T050304         A_31   
315  TRAIN_315        2   0.535965  2022-07-31 14:53  T050304         A_31   
556  TRAIN_556        2   0.560568   2022-08-28 2:54  T050304         A_31   
557  TRAIN_557        2   0.559463   2022-08-28 3:03  T050304         A_31   
562  TRAIN_562        2   0.541429   2022-09-03 1:14  T050304         A_31   
563  TRAIN_563        2   0.537465   2022-09-03 1:22  T050304         A_31   
564  TRAIN_564        2   0.540106   2022-09-03 1:30  T050304         A_31   

     X_1  X_2  X_3  X_4  ...  X_2866  X_2867  X_2868  X_2869  X_2870  X_2871  \
21   NaN  NaN  NaN  NaN  ...   53.49   57.13   50.56   56.59   65.12     1.0   
79   NaN  NaN  NaN  NaN  ...   53.49   55.59   60.00   58.18   66.79     1.0   
80   NaN  NaN  NaN  NaN  ...   55.89   63.42   42.71   57.03   66.20     1.0   
188  NaN  NaN  NaN  NaN  ...   53.49   57.13   50.56   56.59   65.12     1.0   
314  NaN  NaN  NaN  NaN  ...   53.49   58.06   33.43   54.71   61.93     1.0   
315  NaN  NaN  NaN  NaN  ...   56.71   61.84   40.60   56.59   65.12     1.0   
556  NaN  NaN  NaN  NaN  ...   53.49   58.76   42.22   57.92   66.18     1.0   
557  NaN  NaN  NaN  NaN  ...   50.87   57.13   53.07   56.21   67.22     1.0   
562  NaN  NaN  NaN  NaN  ...   56.38   57.10   51.49   56.08   64.40     1.0   
563  NaN  NaN  NaN  NaN  ...   49.92   54.66   50.56   48.63   62.17     1.0   
564  NaN  NaN  NaN  NaN  ...   57.94   55.84   53.24   56.72   64.17     1.0   

     X_2872  X_2873  X_2874  X_2875  
21      NaN     NaN     NaN     NaN  
79      NaN     NaN     NaN     NaN  
80      NaN     NaN     NaN     NaN  
188     NaN     NaN     NaN     NaN  
314     NaN     NaN     NaN     NaN  
315     NaN     NaN     NaN     NaN  
556     NaN     NaN     NaN     NaN  
557     NaN     NaN     NaN     NaN  
562     NaN     NaN     NaN     NaN  
563     NaN     NaN     NaN     NaN  
564     NaN     NaN     NaN     NaN  

[11 rows x 2881 columns]

In [12]:
train_T050304= pd.concat([train_T050304_0,train_T050304_1,train_T050304_2]).sort_values(by='PRODUCT_ID')
train_T050304

PRODUCT_ID  Y_Class  Y_Quality         TIMESTAMP     LINE PRODUCT_CODE  \
0    TRAIN_000        1   0.533433   2022-06-13 5:14  T050304         A_31   
2    TRAIN_002        1   0.531267   2022-06-13 5:30  T050304         A_31   
4    TRAIN_004        1   0.531590   2022-06-13 5:47  T050304         A_31   
6    TRAIN_006        1   0.533665   2022-06-13 6:03  T050304         A_31   
8    TRAIN_008        1   0.531821   2022-06-13 6:19  T050304         A_31   
..         ...      ...        ...               ...      ...          ...   
581  TRAIN_581        0   0.523465   2022-09-05 8:18  T050304         A_31   
582  TRAIN_582        0   0.522233   2022-09-05 8:26  T050304         A_31   
583  TRAIN_583        0   0.522340   2022-09-05 8:34  T050304         A_31   
594  TRAIN_594        0   0.524022  2022-09-08 22:38  T050304         A_31   
595  TRAIN_595        0   0.521289  2022-09-08 22:47  T050304         A_31   

     X_1  X_2  X_3  X_4  ...  X_2866  X_2867  X_2868  X_2869  X_2870  X_2871  \
0    NaN  NaN  NaN  NaN  ...  39.340   40.89   32.56   34.09  77.770     1.0   
2    NaN  NaN  NaN  NaN  ...  39.190   36.65   42.47   36.53  78.350     1.0   
4    NaN  NaN  NaN  NaN  ...  38.700   41.89   46.93   33.09  76.970     1.0   
6    NaN  NaN  NaN  NaN  ...  32.500   41.42   38.36   30.83  76.930     1.0   
8    NaN  NaN  NaN  NaN  ...  42.250   43.17   55.60   33.26  78.500     1.0   
..   ...  ...  ...  ...  ...     ...     ...     ...     ...     ...     ...   
581  NaN  NaN  NaN  NaN  ...  54.115   54.71   52.51   56.30  65.035     1.0   
582  NaN  NaN  NaN  NaN  ...  50.880   53.23   52.44   56.28  66.830     1.0   
583  NaN  NaN  NaN  NaN  ...  51.710   59.64   54.61   57.05  63.180     1.0   
594  NaN  NaN  NaN  NaN  ...  49.470   53.07   50.89   55.10  66.490     1.0   
595  NaN  NaN  NaN  NaN  ...  54.115   54.71   52.51   56.30  65.035     1.0   

     X_2872  X_2873  X_2874  X_2875  
0       NaN     NaN     NaN     NaN  
2       NaN     NaN     NaN     NaN  
4       NaN     NaN     NaN     NaN  
6       NaN     NaN     NaN     NaN  
8       NaN     NaN     NaN     NaN  
..      ...     ...     ...     ...  
581     NaN     NaN     NaN     NaN  
582     NaN     NaN     NaN     NaN  
583     NaN     NaN     NaN     NaN  
594     NaN     NaN     NaN     NaN  
595     NaN     NaN     NaN     NaN  

[78 rows x 2881 columns]

In [13]:
train_T050307_0 = train_T050307[train_T050307['Y_Class'] == 0]
train_T050307_0 = train_T050307_0.fillna(train_T050307_0.median())

train_T050307_1 = train_T050307[train_T050307['Y_Class'] == 1]
train_T050307_1 = train_T050307_1.fillna(train_T050307_1.median())

train_T050307_2 = train_T050307[train_T050307['Y_Class'] == 2]
train_T050307_2 = train_T050307_2.fillna(train_T050307_2.median())

train_T050307= pd.concat([train_T050307_0,train_T050307_1,train_T050307_2]).sort_values(by='PRODUCT_ID')

<ipython-input-13-1d8bd5800171>:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_T050307_0 = train_T050307_0.fillna(train_T050307_0.median())
<ipython-input-13-1d8bd5800171>:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_T050307_1 = train_T050307_1.fillna(train_T050307_1.median())
<ipython-input-13-1d8bd5800171>:8: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_T050307_2 = train_T050307_2.fillna(train_T050307_2.median())


In [14]:
train_T010306_0 = train_T010306[train_T010306['Y_Class'] == 0]
train_T010306_0 = train_T010306_0.fillna(train_T010306_0.median())

train_T010306_1 = train_T010306[train_T010306['Y_Class'] == 1]
train_T010306_1 = train_T010306_1.fillna(train_T010306_1.median())

train_T010306_2 = train_T010306[train_T010306['Y_Class'] == 2]
train_T010306_2 = train_T010306_2.fillna(train_T010306_2.median())

train_T010306= pd.concat([train_T010306_0,train_T010306_1,train_T010306_2]).sort_values(by='PRODUCT_ID')

<ipython-input-14-3e798069e448>:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_T010306_0 = train_T010306_0.fillna(train_T010306_0.median())
<ipython-input-14-3e798069e448>:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_T010306_1 = train_T010306_1.fillna(train_T010306_1.median())
<ipython-input-14-3e798069e448>:8: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_T010306_2 = train_T010306_2.fillna(train_T010306_2.median())


In [15]:
train_T010305_0 = train_T010305[train_T010305['Y_Class'] == 0]
train_T010305_0 = train_T010305_0.fillna(train_T010305_0.median())

train_T010305_1 = train_T010305[train_T010305['Y_Class'] == 1]
train_T010305_1 = train_T010305_1.fillna(train_T010305_1.median())

train_T010305_2 = train_T010305[train_T010305['Y_Class'] == 2]
train_T010305_2 = train_T010305_2.fillna(train_T010305_2.median())

train_T010305= pd.concat([train_T010305_0,train_T010305_1,train_T010305_2]).sort_values(by='PRODUCT_ID')

<ipython-input-15-7204a790f7c4>:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_T010305_0 = train_T010305_0.fillna(train_T010305_0.median())
<ipython-input-15-7204a790f7c4>:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_T010305_1 = train_T010305_1.fillna(train_T010305_1.median())
<ipython-input-15-7204a790f7c4>:8: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_T010305_2 = train_T010305_2.fillna(train_T010305_2.median())


In [16]:
trainA_31 = pd.concat([train_T050304,train_T050307,train_T010306,train_T010305]).sort_values(by='PRODUCT_ID')
trainA_31

PRODUCT_ID  Y_Class  Y_Quality         TIMESTAMP     LINE PRODUCT_CODE  \
0    TRAIN_000        1   0.533433   2022-06-13 5:14  T050304         A_31   
1    TRAIN_001        2   0.541819   2022-06-13 5:22  T050307         A_31   
2    TRAIN_002        1   0.531267   2022-06-13 5:30  T050304         A_31   
3    TRAIN_003        2   0.537325   2022-06-13 5:39  T050307         A_31   
4    TRAIN_004        1   0.531590   2022-06-13 5:47  T050304         A_31   
..         ...      ...        ...               ...      ...          ...   
583  TRAIN_583        0   0.522340   2022-09-05 8:34  T050304         A_31   
584  TRAIN_584        0   0.519519  2022-09-05 11:09  T010305         A_31   
585  TRAIN_585        0   0.515214  2022-09-05 11:17  T010306         A_31   
594  TRAIN_594        0   0.524022  2022-09-08 22:38  T050304         A_31   
595  TRAIN_595        0   0.521289  2022-09-08 22:47  T050304         A_31   

     X_1  X_2  X_3  X_4  ...  X_2866  X_2867  X_2868  X_2869  X_2870  X_2871  \
0    NaN  NaN  NaN  NaN  ...  39.340   40.89   32.56   34.09  77.770     1.0   
1    NaN  NaN  NaN  NaN  ...  38.890   42.82   43.92   35.34  72.550     1.0   
2    NaN  NaN  NaN  NaN  ...  39.190   36.65   42.47   36.53  78.350     1.0   
3    NaN  NaN  NaN  NaN  ...  37.740   39.17   52.17   30.58  71.780     1.0   
4    NaN  NaN  NaN  NaN  ...  38.700   41.89   46.93   33.09  76.970     1.0   
..   ...  ...  ...  ...  ...     ...     ...     ...     ...     ...     ...   
583  NaN  NaN  NaN  NaN  ...  51.710   59.64   54.61   57.05  63.180     1.0   
584  NaN  NaN  NaN  NaN  ...     NaN     NaN     NaN     NaN     NaN     NaN   
585  NaN  NaN  NaN  NaN  ...     NaN     NaN     NaN     NaN     NaN     NaN   
594  NaN  NaN  NaN  NaN  ...  49.470   53.07   50.89   55.10  66.490     1.0   
595  NaN  NaN  NaN  NaN  ...  54.115   54.71   52.51   56.30  65.035     1.0   

     X_2872  X_2873  X_2874  X_2875  
0       NaN     NaN     NaN     NaN  
1       NaN     NaN     NaN     NaN  
2       NaN     NaN     NaN     NaN  
3       NaN     NaN     NaN     NaN  
4       NaN     NaN     NaN     NaN  
..      ...     ...     ...     ...  
583     NaN     NaN     NaN     NaN  
584     NaN     NaN     NaN     NaN  
585     NaN     NaN     NaN     NaN  
594     NaN     NaN     NaN     NaN  
595     NaN     NaN     NaN     NaN  

[249 rows x 2881 columns]

PRODUCT_CODE = T

In [17]:
train_T100304[train_T100304['PRODUCT_CODE'] == 'T_31']
train_T100304[train_T100304['PRODUCT_CODE'] == 'O_31']

PRODUCT_ID  Y_Class  Y_Quality         TIMESTAMP     LINE PRODUCT_CODE  \
569  TRAIN_569        1   0.530533  2022-09-03 18:32  T100304         O_31   
571  TRAIN_571        1   0.525916  2022-09-03 18:48  T100304         O_31   
596  TRAIN_596        1   0.531375  2022-09-08 14:38  T100304         O_31   

      X_1    X_2  X_3   X_4  ...  X_2866  X_2867  X_2868  X_2869  X_2870  \
569   4.0   98.0  0.0  45.0  ...     NaN     NaN     NaN     NaN     NaN   
571   4.0  100.0  0.0  45.0  ...     NaN     NaN     NaN     NaN     NaN   
596  40.0   94.0  0.0  45.0  ...     NaN     NaN     NaN     NaN     NaN   

     X_2871  X_2872  X_2873  X_2874  X_2875  
569     NaN     NaN     NaN     NaN     NaN  
571     NaN     NaN     NaN     NaN     NaN  
596     NaN     NaN     NaN     NaN     NaN  

[3 rows x 2881 columns]

In [18]:
train_T100306[train_T100306['PRODUCT_CODE'] == 'O_31']

PRODUCT_ID  Y_Class  Y_Quality         TIMESTAMP     LINE PRODUCT_CODE  \
570  TRAIN_570        2   0.534951  2022-09-03 18:40  T100306         O_31   
572  TRAIN_572        2   0.535205  2022-09-03 18:56  T100306         O_31   
597  TRAIN_597        1   0.533702  2022-09-08 14:46  T100306         O_31   

      X_1   X_2  X_3   X_4  ...  X_2866  X_2867  X_2868  X_2869  X_2870  \
570   6.0  90.0  0.0  45.0  ...     NaN     NaN     NaN     NaN     NaN   
572   6.0  89.0  0.0  45.0  ...     NaN     NaN     NaN     NaN     NaN   
597  21.0  87.0  0.0  45.0  ...     NaN     NaN     NaN     NaN     NaN   

     X_2871  X_2872  X_2873  X_2874  X_2875  
570     NaN     NaN     NaN     NaN     NaN  
572     NaN     NaN     NaN     NaN     NaN  
597     NaN     NaN     NaN     NaN     NaN  

[3 rows x 2881 columns]

In [19]:
train_T100304_T = train_T100304[train_T100304['PRODUCT_CODE'] == 'T_31']

In [20]:
train_T100304_T_0 = train_T100304_T[train_T100304_T['Y_Class'] == 0]
train_T100304_T_0 = train_T100304_T_0.fillna(train_T100304_T_0.median())

train_T100304_T_1 = train_T100304_T[train_T100304_T['Y_Class'] == 1]
train_T100304_T_1 = train_T100304_T_1.fillna(train_T100304_T_1.median())

train_T100304_T_2 = train_T100304_T[train_T100304_T['Y_Class'] == 2]
train_T100304_T_2 = train_T100304_T_2.fillna(train_T100304_T_2.median())

train_T100304_T = pd.concat([train_T100304_T_0,train_T100304_T_1,train_T100304_T_2]).sort_values(by='PRODUCT_ID')

<ipython-input-20-1edb5b2c7ace>:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_T100304_T_0 = train_T100304_T_0.fillna(train_T100304_T_0.median())
<ipython-input-20-1edb5b2c7ace>:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_T100304_T_1 = train_T100304_T_1.fillna(train_T100304_T_1.median())
<ipython-input-20-1edb5b2c7ace>:8: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_T100304_T_2 = train_T100304_T_2.fillna(train_T100304_T_2.median())


In [21]:
train_T100306_T = train_T100306[train_T100306['PRODUCT_CODE'] == 'T_31']

In [22]:
train_T100306_T_0 = train_T100306_T[train_T100306_T['Y_Class'] == 0]
train_T100306_T_0 = train_T100306_T_0.fillna(train_T100306_T_0.median())

train_T100306_T_1 = train_T100306_T[train_T100306_T['Y_Class'] == 1]
train_T100306_T_1 = train_T100306_T_1.fillna(train_T100306_T_1.median())

train_T100306_T_2 = train_T100306_T[train_T100306_T['Y_Class'] == 2]
train_T100306_T_2 = train_T100306_T_2.fillna(train_T100306_T_2.median())

train_T100306_T = pd.concat([train_T100306_T_0,train_T100306_T_1,train_T100306_T_2]).sort_values(by='PRODUCT_ID')

<ipython-input-22-3b7f4cfa8266>:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_T100306_T_0 = train_T100306_T_0.fillna(train_T100306_T_0.median())
<ipython-input-22-3b7f4cfa8266>:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_T100306_T_1 = train_T100306_T_1.fillna(train_T100306_T_1.median())
<ipython-input-22-3b7f4cfa8266>:8: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_T100306_T_2 = train_T100306_T_2.fillna(train_T100306_T_2.median())


In [23]:
trainT_31 = pd.concat([train_T100304_T,train_T100306_T]).sort_values(by='PRODUCT_ID')
trainT_31

PRODUCT_ID  Y_Class  Y_Quality         TIMESTAMP     LINE PRODUCT_CODE  \
22   TRAIN_022        0   0.517719   2022-06-14 8:53  T100304         T_31   
23   TRAIN_023        0   0.519090   2022-06-14 9:01  T100304         T_31   
25   TRAIN_025        1   0.529362   2022-06-19 9:11  T100304         T_31   
26   TRAIN_026        1   0.531992   2022-06-19 9:20  T100306         T_31   
29   TRAIN_029        1   0.532405  2022-06-19 23:31  T100304         T_31   
..         ...      ...        ...               ...      ...          ...   
589  TRAIN_589        1   0.529510  2022-09-06 18:00  T100306         T_31   
590  TRAIN_590        1   0.529948   2022-09-07 1:01  T100306         T_31   
591  TRAIN_591        1   0.529308   2022-09-07 1:09  T100306         T_31   
592  TRAIN_592        1   0.528349  2022-09-08 14:22  T100304         T_31   
593  TRAIN_593        1   0.526546  2022-09-08 14:30  T100306         T_31   

     X_1    X_2  X_3   X_4  ...  X_2866  X_2867  X_2868  X_2869  X_2870  \
22   2.0  102.0  0.0  45.0  ...     NaN     NaN     NaN     NaN     NaN   
23   2.0  102.0  0.0  45.0  ...     NaN     NaN     NaN     NaN     NaN   
25   2.0   97.0  0.0  45.0  ...     NaN     NaN     NaN     NaN     NaN   
26   2.0   95.0  0.0  45.0  ...     NaN     NaN     NaN     NaN     NaN   
29   2.0  100.0  0.0  45.0  ...     NaN     NaN     NaN     NaN     NaN   
..   ...    ...  ...   ...  ...     ...     ...     ...     ...     ...   
589  1.0   94.0  0.0  45.0  ...     NaN     NaN     NaN     NaN     NaN   
590  1.0   89.0  0.0  45.0  ...     NaN     NaN     NaN     NaN     NaN   
591  1.0   87.0  0.0  45.0  ...     NaN     NaN     NaN     NaN     NaN   
592  2.0   98.0  0.0  45.0  ...     NaN     NaN     NaN     NaN     NaN   
593  2.0   95.0  0.0  45.0  ...     NaN     NaN     NaN     NaN     NaN   

     X_2871  X_2872  X_2873  X_2874  X_2875  
22      NaN     NaN     NaN     NaN     NaN  
23      NaN     NaN     NaN     NaN     NaN  
25      NaN     NaN     NaN     NaN     NaN  
26      NaN     NaN     NaN     NaN     NaN  
29      NaN     NaN     NaN     NaN     NaN  
..      ...     ...     ...     ...     ...  
589     NaN     NaN     NaN     NaN     NaN  
590     NaN     NaN     NaN     NaN     NaN  
591     NaN     NaN     NaN     NaN     NaN  
592     NaN     NaN     NaN     NaN     NaN  
593     NaN     NaN     NaN     NaN     NaN  

[343 rows x 2881 columns]

PRODUCT_CODE O

In [24]:
train_T100304_O = train_T100304[train_T100304['PRODUCT_CODE'] == 'O_31']

train_T100304_O_0 = train_T100304_O[train_T100304_O['Y_Class'] == 0]
train_T100304_O_0 = train_T100304_O_0.fillna(train_T100304_O_0.median())

train_T100304_O_1 = train_T100304_O[train_T100304_O['Y_Class'] == 1]
train_T100304_O_1 = train_T100304_O_1.fillna(train_T100304_O_1.median())

train_T100304_O_2 = train_T100304_O[train_T100304_O['Y_Class'] == 2]
train_T100304_O_2 = train_T100304_O_2.fillna(train_T100304_O_2.median())

train_T100304_O = pd.concat([train_T100304_O_0,train_T100304_O_1,train_T100304_O_2]).sort_values(by='PRODUCT_ID')

<ipython-input-24-c7f663980ac4>:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_T100304_O_1 = train_T100304_O_1.fillna(train_T100304_O_1.median())


In [25]:
train_T100306_O = train_T100306[train_T100306['PRODUCT_CODE'] == 'O_31']

train_T100306_O_0 = train_T100306_O[train_T100306_O['Y_Class'] == 0]
train_T100306_O_0 = train_T100306_O_0.fillna(train_T100306_O_0.median())

train_T100306_O_1 = train_T100306_O[train_T100306_O['Y_Class'] == 1]
train_T100306_O_1 = train_T100306_O_1.fillna(train_T100306_O_1.median())

train_T100306_O_2 = train_T100306_O[train_T100306_O['Y_Class'] == 2]
train_T100306_O_2 = train_T100306_O_2.fillna(train_T100306_O_2.median())

train_T100306_O = pd.concat([train_T100306_O_0,train_T100306_O_1,train_T100306_O_2]).sort_values(by='PRODUCT_ID')

<ipython-input-25-57b6774d6b31>:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_T100306_O_1 = train_T100306_O_1.fillna(train_T100306_O_1.median())
<ipython-input-25-57b6774d6b31>:10: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_T100306_O_2 = train_T100306_O_2.fillna(train_T100306_O_2.median())


In [26]:
trainO_31 = pd.concat([train_T100304_O,train_T100306_O]).sort_values(by='PRODUCT_ID')
trainO_31

PRODUCT_ID  Y_Class  Y_Quality         TIMESTAMP     LINE PRODUCT_CODE  \
569  TRAIN_569        1   0.530533  2022-09-03 18:32  T100304         O_31   
570  TRAIN_570        2   0.534951  2022-09-03 18:40  T100306         O_31   
571  TRAIN_571        1   0.525916  2022-09-03 18:48  T100304         O_31   
572  TRAIN_572        2   0.535205  2022-09-03 18:56  T100306         O_31   
596  TRAIN_596        1   0.531375  2022-09-08 14:38  T100304         O_31   
597  TRAIN_597        1   0.533702  2022-09-08 14:46  T100306         O_31   

      X_1    X_2  X_3   X_4  ...  X_2866  X_2867  X_2868  X_2869  X_2870  \
569   4.0   98.0  0.0  45.0  ...     NaN     NaN     NaN     NaN     NaN   
570   6.0   90.0  0.0  45.0  ...     NaN     NaN     NaN     NaN     NaN   
571   4.0  100.0  0.0  45.0  ...     NaN     NaN     NaN     NaN     NaN   
572   6.0   89.0  0.0  45.0  ...     NaN     NaN     NaN     NaN     NaN   
596  40.0   94.0  0.0  45.0  ...     NaN     NaN     NaN     NaN     NaN   
597  21.0   87.0  0.0  45.0  ...     NaN     NaN     NaN     NaN     NaN   

     X_2871  X_2872  X_2873  X_2874  X_2875  
569     NaN     NaN     NaN     NaN     NaN  
570     NaN     NaN     NaN     NaN     NaN  
571     NaN     NaN     NaN     NaN     NaN  
572     NaN     NaN     NaN     NaN     NaN  
596     NaN     NaN     NaN     NaN     NaN  
597     NaN     NaN     NaN     NaN     NaN  

[6 rows x 2881 columns]

# product code별 학습진행
  - LINE 별 dataset 에서 Y_Class별로 나누어 mean 적용(결측치 없앰)
  - 나머지 존재하는 NaN에 대해서는 -1 대체 (변수가 의미없음을 의미)
  - 변수개수는 줄이지 않았음
  - PRODUCT_CODE O는 6개 존재(Y_Class = 1 4개, Y_Class = 2 2개) -> 이는 분류 보단 회귀 이후 라벨링이 적절해보임.

Dataset:  
- trainA_31  
- trainT_31  
- trainO_31

## test nan 처리에 대해 고민해봐야할듯
- test에도 O는 4개밖에 없어서 O를 맞추는 거는 크게 고민 안해도 될듯
- A와 T위주로 맞추는걸로

In [28]:
testA_31 = test_df[test_df['PRODUCT_CODE'] == 'A_31']
testT_31 = test_df[test_df['PRODUCT_CODE'] == 'T_31']
testO_31 = test_df[test_df['PRODUCT_CODE'] == 'O_31']

In [29]:
trainA_31 = remove_all_nan(trainA_31)
trainT_31 = remove_all_nan(trainT_31)
trainO_31 = remove_all_nan(trainO_31)

In [30]:
trainA_31_x = trainA_31.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE','Y_Class','Y_Quality'])
trainT_31_x = trainT_31.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE','Y_Class','Y_Quality'])
trainO_31_x = trainO_31.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE','Y_Class','Y_Quality'])

In [31]:
trainA_31_x = pd.get_dummies(trainA_31_x)
trainT_31_x = pd.get_dummies(trainT_31_x)
trainO_31_x = pd.get_dummies(trainO_31_x)

In [35]:
trainO_31_x

X_1    X_2  X_3   X_4   X_5  X_6   X_7   X_8   X_9  X_10  ...  \
569   4.0   98.0  0.0  45.0  11.0  0.0  45.0  10.0  31.0   2.0  ...   
570   6.0   90.0  0.0  45.0  10.0  0.0  51.0  10.0  52.0   2.0  ...   
571   4.0  100.0  0.0  45.0  11.0  0.0  45.0  10.0  31.0   2.0  ...   
572   6.0   89.0  0.0  45.0  10.0  0.0  51.0  10.0  52.0   2.0  ...   
596  40.0   94.0  0.0  45.0  11.0  0.0  45.0  10.0  31.0   2.0  ...   
597  21.0   87.0  0.0  45.0  10.0  0.0  61.0  10.0  52.0   2.0  ...   

          X_926  X_927  X_928       X_929  X_930  X_931      X_932  X_933  \
569  155.541935  149.8  327.0  312.018182  298.0   13.6  13.458065   13.3   
570  157.464516  154.5  327.0  312.454545  298.0   13.7  13.454839   13.3   
571  159.864516  148.7  326.0  311.763636  298.0   13.7  13.425806   13.2   
572  161.046667  151.3  326.0  312.537037  298.0   13.7  13.440000   13.2   
596  173.190323  168.6  321.0  303.036364  289.0   13.6  13.406667   13.2   
597  167.087097  161.1  326.0  304.818182  289.0   13.6  13.380000   13.2   

     LINE_T100304  LINE_T100306  
569             1             0  
570             0             1  
571             1             0  
572             0             1  
596             1             0  
597             0             1  

[6 rows x 669 columns]

## train set iterativeImputer 적용

In [36]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

def iterativeimputer_subset(input_df,target_df,LINE_NUM):
  imputer = IterativeImputer(random_state=1234)

  cols = input_df.columns[:-1 * LINE_NUM]
  key_cols = input_df.columns[-1 * LINE_NUM:]

  total_subset = target_df[key_cols]
  interval = 30
  
  for i in range(0,len(cols)//interval):
    print(i,'번째 merge 진행중 ...')
    subset = pd.concat([input_df[key_cols], input_df[cols[i*interval : (i*interval)+interval]]], axis=1)
    imputer.fit(subset)

    target_subset = pd.concat([target_df[key_cols], target_df[cols[i*interval : (i*interval)+interval]]], axis=1)
    impute_subset = pd.DataFrame(imputer.transform(target_subset), columns=target_subset.columns)
    impute_subset = impute_subset.drop(key_cols,axis=1)

    print("기존 total : ", total_subset.shape, "기존 impute_subset : ", impute_subset.shape)
    total_subset = pd.concat([total_subset.reset_index(drop=True), impute_subset.reset_index(drop=True)], axis=1)
    print("병합 total : ", total_subset.shape)
    print('------------------------------------------------------')

  subset = pd.concat([input_df[key_cols], input_df[cols[total_subset.shape[1]-len(input_df.columns) : ]]], axis=1)
  imputer.fit(subset)

  target_subset = pd.concat([target_df[key_cols], target_df[cols[total_subset.shape[1]-len(input_df.columns) : ]]], axis=1)
  impute_subset = pd.DataFrame(imputer.transform(target_subset), columns=target_subset.columns)
  impute_subset = impute_subset.drop(key_cols,axis=1)

  print("기존 total : ", total_subset.shape, "기존 impute_subset : ", impute_subset.shape)
  total_subset = pd.concat([total_subset.reset_index(drop=True), impute_subset.reset_index(drop=True)], axis=1)
  print("병합 total : ", total_subset.shape)
  print('------------------------------------------------------')

  return total_subset

In [37]:
trainA_31_x = iterativeimputer_subset(trainA_31_x,trainA_31_x,4)
trainT_31_x = iterativeimputer_subset(trainT_31_x,trainT_31_x,2)
trainO_31_x = iterativeimputer_subset(trainO_31_x,trainO_31_x,2)

0 번째 merge 진행중 ...


/usr/local/lib/python3.8/dist-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


기존 total :  (249, 4) 기존 impute_subset :  (249, 30)
병합 total :  (249, 34)
------------------------------------------------------
1 번째 merge 진행중 ...
기존 total :  (249, 34) 기존 impute_subset :  (249, 30)
병합 total :  (249, 64)
------------------------------------------------------
2 번째 merge 진행중 ...


/usr/local/lib/python3.8/dist-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


기존 total :  (249, 64) 기존 impute_subset :  (249, 30)
병합 total :  (249, 94)
------------------------------------------------------
3 번째 merge 진행중 ...
기존 total :  (249, 94) 기존 impute_subset :  (249, 30)
병합 total :  (249, 124)
------------------------------------------------------
4 번째 merge 진행중 ...
기존 total :  (249, 124) 기존 impute_subset :  (249, 30)
병합 total :  (249, 154)
------------------------------------------------------
5 번째 merge 진행중 ...
기존 total :  (249, 154) 기존 impute_subset :  (249, 30)
병합 total :  (249, 184)
------------------------------------------------------
6 번째 merge 진행중 ...
기존 total :  (249, 184) 기존 impute_subset :  (249, 30)
병합 total :  (249, 214)
------------------------------------------------------
7 번째 merge 진행중 ...
기존 total :  (249, 214) 기존 impute_subset :  (249, 30)
병합 total :  (249, 244)
------------------------------------------------------
8 번째 merge 진행중 ...
기존 total :  (249, 244) 기존 impute_subset :  (249, 30)
병합 total :  (249, 274)
---------------------------

...

기존 total :  (249, 2104) 기존 impute_subset :  (249, 16)
병합 total :  (249, 2120)
------------------------------------------------------
0 번째 merge 진행중 ...
기존 total :  (343, 2) 기존 impute_subset :  (343, 30)
병합 total :  (343, 32)
------------------------------------------------------
1 번째 merge 진행중 ...
기존 total :  (343, 32) 기존 impute_subset :  (343, 30)
병합 total :  (343, 62)
------------------------------------------------------
2 번째 merge 진행중 ...
기존 total :  (343, 62) 기존 impute_subset :  (343, 30)
병합 total :  (343, 92)
------------------------------------------------------
3 번째 merge 진행중 ...
기존 total :  (343, 92) 기존 impute_subset :  (343, 30)
병합 total :  (343, 122)
------------------------------------------------------
4 번째 merge 진행중 ...


/usr/local/lib/python3.8/dist-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


기존 total :  (343, 122) 기존 impute_subset :  (343, 30)
병합 total :  (343, 152)
------------------------------------------------------
5 번째 merge 진행중 ...
기존 total :  (343, 152) 기존 impute_subset :  (343, 30)
병합 total :  (343, 182)
------------------------------------------------------
6 번째 merge 진행중 ...
기존 total :  (343, 182) 기존 impute_subset :  (343, 30)
병합 total :  (343, 212)
------------------------------------------------------
7 번째 merge 진행중 ...
기존 total :  (343, 212) 기존 impute_subset :  (343, 30)
병합 total :  (343, 242)
------------------------------------------------------
8 번째 merge 진행중 ...
기존 total :  (343, 242) 기존 impute_subset :  (343, 30)
병합 total :  (343, 272)
------------------------------------------------------
9 번째 merge 진행중 ...
기존 total :  (343, 272) 기존 impute_subset :  (343, 30)
병합 total :  (343, 302)
------------------------------------------------------
10 번째 merge 진행중 ...
기존 total :  (343, 302) 기존 impute_subset :  (343, 30)
병합 total :  (343, 332)
-----------------------

/usr/local/lib/python3.8/dist-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


기존 total :  (6, 122) 기존 impute_subset :  (6, 30)
병합 total :  (6, 152)
------------------------------------------------------
5 번째 merge 진행중 ...
기존 total :  (6, 152) 기존 impute_subset :  (6, 30)
병합 total :  (6, 182)
------------------------------------------------------
6 번째 merge 진행중 ...
기존 total :  (6, 182) 기존 impute_subset :  (6, 30)
병합 total :  (6, 212)
------------------------------------------------------
7 번째 merge 진행중 ...
기존 total :  (6, 212) 기존 impute_subset :  (6, 30)
병합 total :  (6, 242)
------------------------------------------------------
8 번째 merge 진행중 ...
기존 total :  (6, 242) 기존 impute_subset :  (6, 30)
병합 total :  (6, 272)
------------------------------------------------------
9 번째 merge 진행중 ...
기존 total :  (6, 272) 기존 impute_subset :  (6, 30)
병합 total :  (6, 302)
------------------------------------------------------
10 번째 merge 진행중 ...
기존 total :  (6, 302) 기존 impute_subset :  (6, 30)
병합 total :  (6, 332)
------------------------------------------------------
11 번째 merg

## test set iterativeImputer 적용

In [44]:
testA_31_x = pd.concat([testA_31[trainA_31_x.columns[4:]], testA_31['LINE']], axis=1)
testA_31_x = pd.get_dummies(testA_31_x)

testT_31_x = pd.concat([testT_31[trainT_31_x.columns[2:]], testT_31['LINE']], axis=1)
testT_31_x = pd.get_dummies(testT_31_x)

testO_31_x = pd.concat([testO_31[trainO_31_x.columns[2:]], testO_31['LINE']], axis=1)
testO_31_x = pd.get_dummies(testO_31_x)
testO_31_x

X_1    X_2  X_3   X_4   X_5  X_6   X_7   X_8   X_9  X_10  ...  \
138    4.0   98.0  0.0  45.0  10.0  0.0  45.0  10.0  31.0   2.0  ...   
256  154.0   97.0  0.0  45.0  11.0  0.0  45.0  10.0  31.0   2.0  ...   
257  146.0   94.0  0.0  45.0  10.0  0.0  67.0  11.0  52.0   2.0  ...   
287  133.0  100.0  0.0  45.0  10.0  0.0  45.0  10.0  31.0   2.0  ...   

          X_926  X_927  X_928       X_929  X_930  X_931      X_932  X_933  \
138  149.832258  142.0  326.0  310.490909  289.0   13.7  13.430000   13.2   
256  171.522581  163.3  326.0  305.890909  289.0   13.6  13.396667   13.2   
257  168.043333  161.9  327.0  309.745454  291.0   13.6  13.350000   13.2   
287  174.393548  164.0  326.0  310.000000  289.0   13.6  13.396774   13.2   

     LINE_T100304  LINE_T100306  
138             1             0  
256             1             0  
257             0             1  
287             1             0  

[4 rows x 669 columns]

In [45]:
testA_31_x = iterativeimputer_subset(trainA_31_x,testA_31_x,4)
testT_31_x = iterativeimputer_subset(trainT_31_x,testT_31_x,2)
testO_31_x = iterativeimputer_subset(trainO_31_x,testO_31_x,2)

0 번째 merge 진행중 ...
기존 total :  (67, 4) 기존 impute_subset :  (67, 30)
병합 total :  (67, 34)
------------------------------------------------------
1 번째 merge 진행중 ...
기존 total :  (67, 34) 기존 impute_subset :  (67, 30)
병합 total :  (67, 64)
------------------------------------------------------
2 번째 merge 진행중 ...
기존 total :  (67, 64) 기존 impute_subset :  (67, 30)
병합 total :  (67, 94)
------------------------------------------------------
3 번째 merge 진행중 ...
기존 total :  (67, 94) 기존 impute_subset :  (67, 30)
병합 total :  (67, 124)
------------------------------------------------------
4 번째 merge 진행중 ...
기존 total :  (67, 124) 기존 impute_subset :  (67, 30)
병합 total :  (67, 154)
------------------------------------------------------
5 번째 merge 진행중 ...
기존 total :  (67, 154) 기존 impute_subset :  (67, 30)
병합 total :  (67, 184)
------------------------------------------------------
6 번째 merge 진행중 ...
기존 total :  (67, 184) 기존 impute_subset :  (67, 30)
병합 total :  (67, 214)
----------------------------------

Dataset:  
- trainA_31  
- trainT_31  
- trainO_31  

PRODUCT_CODE 별 존재하는 NaN에 대해서는 iterativeimputer 적용
  - 모두 NaN인 칼럼은 삭제(train)
  - test는 train 칼럼 그대로 이용

test 결측치
  - train에 iterativeimputer fit, test에 transform 적용

Dataset:
- trainA_31_x, testA_31_x
- trainT_31_x, testT_31_x 
- trainO_31_x, testO_31_x




In [46]:
# classification
trainA_31_y_c = trainA_31['Y_Class']
trainT_31_y_c = trainT_31['Y_Class']
trainO_31_y_c = trainO_31['Y_Class']

# regression
trainA_31_y_r = trainA_31['Y_Quality']
trainT_31_y_r = trainT_31['Y_Quality']
trainO_31_y_r = trainO_31['Y_Quality']

## model

In [48]:
!pip install --target=$my_path catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 11.3 MB/s eta 0:00:00


In [49]:
from catboost import *

In [50]:
model = CatBoostRegressor(random_state=1234,verbose=500,iterations=1500,learning_rate=0.033)
model.fit(trainA_31_x, trainA_31_y_r)
pred_a = model.predict(testA_31_x)

0:	learn: 0.0098236	total: 208ms	remaining: 5m 11s
500:	learn: 0.0005345	total: 1m 3s	remaining: 2m 6s
1000:	learn: 0.0000449	total: 1m 56s	remaining: 58.2s
1499:	learn: 0.0000048	total: 2m 48s	remaining: 0us


In [51]:
#model = CatBoostRegressor(random_state=110,verbose=500,iterations=500)
model.fit(trainT_31_x, trainT_31_y_r)
pred_t = model.predict(testT_31_x)

0:	learn: 0.0047003	total: 42.1ms	remaining: 1m 3s
500:	learn: 0.0004210	total: 17.3s	remaining: 34.6s
1000:	learn: 0.0000437	total: 36s	remaining: 17.9s
1499:	learn: 0.0000055	total: 52.8s	remaining: 0us


In [52]:
#model = CatBoostRegressor(random_state=110,verbose=500,iterations=500)
model.fit(trainO_31_x, trainO_31_y_r)
pred_o = model.predict(testO_31_x)

0:	learn: 0.0031679	total: 4.5ms	remaining: 6.75s
500:	learn: 0.0000469	total: 1.82s	remaining: 3.64s
1000:	learn: 0.0000007	total: 3.72s	remaining: 1.85s
1499:	learn: 0.0000000	total: 8.08s	remaining: 0us


In [53]:
testA_31['Y_quanlity'] = pred_a
testT_31['Y_quanlity'] = pred_t
testO_31['Y_quanlity'] = pred_o

<ipython-input-53-30f484713f3e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testA_31['Y_quanlity'] = pred_a
<ipython-input-53-30f484713f3e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testT_31['Y_quanlity'] = pred_t
<ipython-input-53-30f484713f3e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

In [54]:
testA_31['Y_Class'] = 1
testT_31['Y_Class'] = 1
testO_31['Y_Class'] = 1

<ipython-input-54-dd33eabf9461>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testA_31['Y_Class'] = 1
<ipython-input-54-dd33eabf9461>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testT_31['Y_Class'] = 1
<ipython-input-54-dd33eabf9461>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

In [55]:
testA_31.loc[(testA_31['Y_quanlity']<0.52507), 'Y_Class'] = 0
testA_31.loc[(testA_31['Y_quanlity']>0.5349), 'Y_Class'] = 2

testT_31.loc[(testT_31['Y_quanlity']<0.52507), 'Y_Class'] = 0
testT_31.loc[(testT_31['Y_quanlity']>0.5349), 'Y_Class'] = 2

testO_31.loc[(testO_31['Y_quanlity']<0.52507), 'Y_Class'] = 0
testO_31.loc[(testO_31['Y_quanlity']>0.5349), 'Y_Class'] = 2

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [56]:
submita = pd.read_csv('/content/drive/MyDrive/LGaimers/sample_submission.csv')
submitt = pd.read_csv('/content/drive/MyDrive/LGaimers/sample_submission.csv')
submito = pd.read_csv('/content/drive/MyDrive/LGaimers/sample_submission.csv')

In [57]:
submita = pd.merge(submita[['PRODUCT_ID']],testA_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submitt = pd.merge(submitt[['PRODUCT_ID']],testT_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submito = pd.merge(submito[['PRODUCT_ID']],testO_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')

In [58]:
pd.concat([submita,submitt,submito]).sort_values(by='PRODUCT_ID').to_csv('cat + mice.csv',index=False)

## 검증

In [66]:
result = pd.read_csv('/content/cat + mice.csv')
cat2 = pd.read_csv('/content/캣2_5.csv')

In [67]:
result['Y_Class'].value_counts()

1    305
0      4
2      1
Name: Y_Class, dtype: int64

In [68]:
from collections import Counter
Counter(result['Y_Class'] != cat2['Y_Class'])

Counter({False: 278, True: 32})